# **PARSING FUNCTIONS**  

Read the code if you want, if not just run the next cell.

In [2]:
import xml.etree.ElementTree as et
import pandas as pd
from IPython.core import display as ICD
from pandas import json_normalize
import pandas as pd
import tqdm as tqdm
import os
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from datetime import datetime
import time

#@title
types_dict={1:"Pass",2:"Offside Pass",3:"Take On",4:"Foul",5:"Out",6:"Corner Awarded",7:"Tackle",8:"Interception",9:"Turnover",10:"Save",11:"Claim",12:"Clearance",13:"Miss",14:"Post",15:"Attempt Saved",16:"Goal",17:"Card",18:"Player off",19:"Player on",20:"Player retired",21:"Player returns",22:"Player becomes goalkeeper",23:"Goalkeeper becomes player",24:"Condition change",25:"Official change",27:"Start delay",28:"End delay",30:"End",31:"Picked an orange",32:"Start",34:"Team set up",35:"Player changed position",36:"Player changed Jersey number",37:"Collection End",38:"Temp_Goal",39:"Temp_Attempt",40:"Formation change",41:"Punch",42:"Good Skill",43:"Deleted event",44:"Aerial",45:"Challenge",47:"Rescinded card",49:"Ball recovery",50:"Dispossessed",51:"Error",52:"Keeper pick-up",53:"Cross not claimed",54:"Smother",55:"Offside provoked",56:"Shield ball opp",57:"Foul throw-in",58:"Penalty faced",59:"Keeper Sweeper",60:"Chance missed",61:"Ball touch",63:"Temp_Save",64:"Resume",65:"Contentious referee decision",67:"50/50",68:"Referee drop ball",69:"Failed To Block",72:"Caught offside",73:"Other Ball Contact",74:"Blocked pass"};
types = pd.DataFrame.from_dict(types_dict,orient='index').reset_index()
types.columns=["type_id","event_name"]

qualifiers_dict={1:"Long ball",2:"Cross",3:"Head pass",4:"Through ball",5:"Free kick taken",6:"Corner taken",7:"Players caught offside",8:"Goal disallowed",9:"Penalty",10:"Hand",11:"6-seconds violation",12:"Dangerous play",13:"Foul",14:"Last line",15:"Head",16:"Small box-centre",17:"Box-centre",18:"Out of box-centre",19:"35+ centre",20:"Right footed",21:"Other body part",22:"Regular play",23:"Fast break",24:"Set piece",25:"From corner",26:"Free kick",28:"Own goal",29:"Assisted",30:"Involved",31:"Yellow Card",32:"Second yellow",33:"Red card",34:"Referee abuse",35:"Argument",36:"Fight",37:"Time wasting",38:"Excessive celebration",39:"Crowd interaction",40:"Other reason",41:"Injury",42:"Tactical",44:"Player position",49:"Attendance figure",50:"Official position",51:"Official ID",53:"Injured player id",54:"End cause",55:"Related event ID",56:"Zone",57:"End type",59:"Jersey number",60:"Small box-right",61:"Small box-left",62:"Box-deep right",63:"Box-right",64:"Box-left",65:"Box-deep left",66:"Out of box-deep right",67:"Out of box-right",68:"Out of box-left",69:"Out of box-deep left",70:"35+ right",71:"35+ left",72:"Left footed",73:"Left",74:"High",75:"Right",76:"Low left",77:"High left",78:"Low centre",79:"High centre",80:"Low right",81:"High right",82:"Blocked",83:"Close left",84:"Close right",85:"Close high",86:"Close left and high",87:"Close right and high",88:"High claim",89:"1 on 1",90:"Deflected save",91:"Dive and deflect",92:"Catch",93:"Dive and catch",94:"Def block",95:"Back pass",96:"Corner situation",97:"Direct free",100:"Six yard blocked",101:"Saved off line",102:"Goal mouth y co-ordinate",103:"Goal mouth z co-ordinate",106:"Attacking Pass",107:"Throw-in",108:"Volley",109:"Overhead",110:"Half Volley",111:"Diving Header",112:"Scramble",113:"Strong",114:"Weak",115:"Rising",116:"Dipping",117:"Lob",118:"One Bounce",119:"Few Bounces",120:"Swerve Left",121:"Swerve Right",122:"Swerve Moving",123:"Keeper Throw",124:"Goal Kick",127:"Direction of play",128:"Punch",130:"Team formation",131:"Team player formation",132:"Dive",133:"Deflection",134:"Far Wide Left",135:"Far Wide Right",136:"Keeper Touched",137:"Keeper Saved",138:"Hit Woodwork",139:"Own Player",140:"Pass End X",141:"Pass End Y",144:"Deleted event type",145:"Formation slot",146:"Blocked x co-ordinate",147:"Blocked y co-ordinate",153:"Not past goal line",154:"Intentional assist",155:"Chipped",156:"Lay-off",157:"Launch",158:"Persistent infringement",159:"Foul and abusive language",160:"Throw-in set piece",161:"Encroachment",162:"Leaving field",163:"Entering field",164:"Spitting",165:"Professional foul",166:"Handling on the line",167:"Out of play",168:"Flick-on",169:"Leading to attempt",170:"Leading to goal",171:"Rescinded card",172:"No impact on timing",173:"Parried safe",174:"Parried danger",175:"Fingertip",176:"Caught",177:"Collected",178:"Standing",179:"Diving",180:"Stooping",181:"Reaching",182:"Hands",183:"Feet",184:"Dissent",185:"Blocked cross",186:"Scored",187:"Saved",188:"Missed",189:"Player not visible",190:"From shot off target",191:"Off the ball foul",192:"Block by hand",194:"Captain",195:"Pull Back",196:"Switch of play",197:"Team kit",198:"GK hoof",199:"Gk kick from hands",200:"Referee stop",201:"Referee delay",202:"Weather problem",203:"Crowd trouble",204:"Fire",205:"Object thrown on pitch",206:"Spectator on pitch",207:"Awaiting officials decision",208:"Referee Injury",209:"Game end",210:"Assist",211:"Overrun",212:"Length",213:"Angle",214:"Big Chance",215:"Individual Play",216:"2nd related event ID",217:"2nd assisted",218:"2nd assist",219:"Players on both posts",220:"Player on near post",221:"Player on far post",222:"No players on posts",223:"In-swinger",224:"Out-swinger",225:"Straight",226:"Suspended",227:"Resume",228:"Own shot blocked",229:"Post-match complete",230:"GK X Coordinate",231:"GK Y Coordinate",232:"Unchallenged"};
qualifiers = pd.DataFrame.from_dict(qualifiers_dict,orient='index').reset_index()
qualifiers.columns = ["qualifier_id","description"]

qualifiers_dict2 = {str(key): str(value) for key, value in qualifiers_dict.items()}


def parsef24(F24file):
  tree = et.ElementTree(file = F24file)
  games = tree.getroot()
  gameinfo = games.findall('Game')
  gameinfo = gameinfo[0]
  game_id = gameinfo.get('id')
  home_team_id = gameinfo.get('home_team_id')
  home_team_name = gameinfo.get('home_team_name')
  away_team_id = gameinfo.get('away_team_id')
  away_team_name = gameinfo.get('away_team_name')
  competition_id = gameinfo.get('competition_id')
  competition_name = gameinfo.get('competition_name')
  season_id = gameinfo.get('season_id')

  global match_events
  match_events = pd.DataFrame()
  for game in games:
      for event in game:
          evento = json_normalize(event.attrib)
          mylist = []
          for q in event:
            mylist.append(q.attrib)
          evento["qualifiers"] = ["a"]
          evento.at[0,"qualifiers"] = mylist
          match_events = pd.concat([match_events,evento])
  match_events[["id","event_id","type_id","period_id","min","sec"]] = match_events[["id","event_id","type_id","period_id","min","sec"]].astype(int)
  match_events[["y","x"]] = match_events[["y","x"]].astype(float)
  match_events = pd.merge(match_events,types, on="type_id", how = "left")
  match_events = match_events[ ['id',"event_id","type_id", "event_name" ]+ [ col for col in match_events.columns if col not in ['id',"event_id","type_id", "event_name" ] ] ]
  match_events['competition_id'] = competition_id
  match_events['game_id'] = game_id
  match_events['home_team_id'] = home_team_id
  match_events['home_team_name'] = home_team_name
  match_events['away_team_id'] = away_team_id
  match_events['away_team_name'] = away_team_name
  match_events['competition_id'] = competition_id
  match_events['competition_name'] = competition_name
  match_events['season_id'] = season_id
  match_events['competition_id'] = competition_id

def explode_event(nome_df,id_evento,mytresh):
  global exploded_df
  exploded_df = pd.DataFrame()
  nome_df=nome_df[nome_df["type_id"]==id_evento]
  allqualifiers = pd.DataFrame()
  for i in range(len(nome_df)):
    qualifiers_detailed = json_normalize(nome_df.qualifiers.iloc[i]).T.reset_index().drop("index",axis=1).fillna("yes")
    qualifiers_detailed.columns = list(qualifiers_detailed.iloc[1])
    qualifiers_detailed = qualifiers_detailed.drop(labels=[0,1], axis=0)
    qualifiers_detailed.rename(columns=qualifiers_dict2,inplace=True)
    qualifiers_detailed["id"] = nome_df.id.iloc[i]
    allqualifiers = pd.concat([allqualifiers,qualifiers_detailed])
  exploded_df = pd.concat([exploded_df,pd.merge(nome_df.drop("qualifiers", axis=1), allqualifiers.reindex(sorted(allqualifiers.columns), axis=1).reset_index()\
                                            .drop("index", axis=1).dropna(thresh=len(allqualifiers)*mytresh, axis=1), on="id", how="outer")]).fillna("-")

#**PLEASE READ**:


**types** - DataFrame with all the IDs of event types and their respective names. Print 'types.head()' to get an idea of the content.


**qualifiers** - DataFrame with all the IDs of qualifiers and their respective names.
Print 'qualifiers.head()' to get an idea of the content



---







**parsef24** - Function that transforms an Opta F24 XML feed into a pandas DataFrame named match_events. To be executed correctly, it should receive the XML file's name as a parameter (which should already be loaded into Google Colab).
Print 'match_events.head(15)' to get an idea of the content.



---




**explode_event** - Function that takes the DataFrame 'match_events' (or another one filtered by you) and the type of event you want, "exploding" all qualifiers into columns to create a new DataFrame named "exploded_df". The last parameter is the threshold that defines the minimum percentage of non-null values that the columns must contain. If it is 1, it will return only columns with no NaN values, while if it is 0, it returns all columns.


Example:
```
explode_event(match_events, 1, 0.15)
```

# **METRIC CREATION**

## Metric Specifications

We want to calculate the average time a team takes to recover the ball.

Metric Specifications:
* Calculate the time to recover the ball since they lost it
* Count the total ball recoveries
* Calculate the average time for each team

We want to  create a table with columns:
* Team ID
* Team Name
* Ball recovery time

## Functions

In [12]:
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")


### Clean the data

clean_data function retrieves only rows that are ball recoveries.
Also, there are sometimes where the data indicates that a team recovers the ball from themselves, we want to recover the ball from the opposite team

In [11]:
def clean_data():
    
    #We get all the ball recoveries from the match_events dataframe
    ball_recoveries = match_events[(match_events["event_name"] == "Ball recovery")]

    #reset the index
    ball_recoveries.reset_index(inplace=True, drop=True)

    indices_to_drop = []

    # Iterate through the DataFrame and find the times where the ball recovery is from the same team
    for index in range(1, len(ball_recoveries)-1):
        if (ball_recoveries.iloc[index-1]["team_id"] == ball_recoveries.iloc[index]["team_id"]):
            indices_to_drop.append(index)

    # Drop all the rows at once
    ball_recoveries.drop(indices_to_drop, inplace=True)
    ball_recoveries.reset_index(inplace=True, drop=True)

    return ball_recoveries

### Ball recovery time calculator

Function that calculates the time it takes a team to recover the ball, it also adds the data into the teams_df dataframe that contains all ball recovery times from all matches and teams

In [10]:
def recovery_time_calculation(ball_recoveries, teams_df):
    # For loop to iterate each ball recovery
    for index, ball_recovery in ball_recoveries.iterrows():

        # Convert index to integer
        index = int(index)
        
        # Find the seconds where the ball was last recovered
        if index == 0:
            prev_min = 0
        else:
            prev_min = (ball_recoveries.iloc[index-1]["min"] * 60) + ball_recoveries.iloc[index-1]["sec"]
        
        # Calculate the time to recover the ball based on the previous ball recovery time
        recovery_time = ((ball_recovery["min"] * 60) + ball_recovery["sec"]) - prev_min

        # Check if the recovery was done by the home team or away team
        team_name = ball_recovery["home_team_name"] if ball_recovery["team_id"] == ball_recovery["home_team_id"] else ball_recovery["away_team_name"]
        
        # Create a dictionary for the new row
        data = {'team_name': team_name, 'recovery_time': recovery_time}
        values = pd.DataFrame([data])  # Create a DataFrame from the dictionary
        
        # Concatenate the new row to the results DataFrame
        teams_df = pd.concat([teams_df, values], ignore_index=True)
    
    return teams_df


## Code

In [13]:
# Initialize dataframe that will contain our information
teams_df = pd.DataFrame(columns=['team_name', 'recovery_time'])
teams_df

,team_name,recovery_time


Iterate through all the files in the **content** folder

In [14]:
basepath = "content/"

# iterate through
for file in tqdm(os.listdir(basepath)):
    print(basepath + file)
    parsef24(basepath + file)
    ball_recoveries = clean_data()
    teams_df = recovery_time_calculation(ball_recoveries,teams_df)
    

# Remove any negative values in our data
teams_df = teams_df[teams_df['recovery_time'] >= 0]   
teams_df.info()

  0%|          | 0/333 [00:00<?, ?it/s]

content/f24-23-2022-2301589-eventdetails.xml


  0%|          | 1/333 [00:03<21:22,  3.86s/it]

content/f24-23-2022-2301752-eventdetails.xml


  1%|          | 2/333 [00:07<19:39,  3.56s/it]

...

content/f24-23-2022-2301885-eventdetails.xml


100%|█████████▉| 332/333 [19:31<00:03,  3.18s/it]

content/f24-23-2022-2301863-eventdetails.xml


100%|██████████| 333/333 [19:34<00:00,  3.53s/it]

<class 'pandas.core.frame.DataFrame'>
Index: 24522 entries, 0 to 24660
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team_name      24522 non-null  object
 1   recovery_time  24522 non-null  object
dtypes: object(2)
memory usage: 574.7+ KB


Now we group the dataframe by teams and by their average recovery time
We sort by descending recovery times.

In [15]:
# Use the groupby function to find the average recovery time by team
final_df = teams_df.groupby('team_name')['recovery_time'].mean()
final_df = pd.DataFrame(final_df).sort_values("recovery_time", ascending=True)

#Round recovery_time to a single decimal place
final_df["recovery_time"] = final_df["recovery_time"].apply(lambda x: round(x, 1))
final_df = final_df.reset_index()

final_df

,team_name,recovery_time
0,Athletic Club,64.6
1,Barcelona,67.7
2,Atlético de Madrid,69.1
3,Valencia CF,70.6
4,Celta de Vigo,72.7
5,Real Madrid,73.8
6,Sevilla,75.1
7,Real Sociedad,75.6
8,Real Valladolid,75.8
9,Rayo Vallecano,77.6
